In [3]:
import handcalcs.render
from handcalcs.decorator import handcalc

# Rentesregning, annuitetsopsparing og -gæld

## Renteomregning - år til måned

$$r_m = (1+r_å)^{1/12} - 1 $$

## Renteomregning - år til kvartal

$$r_k = (1+r_å)^{1/4} - 1 $$

In [4]:
@handcalc(override = "long", jupyter_display=True, precision=8)
def å2m(r):
  r_m = (1 + r)**(1/12) - 1

In [5]:
@handcalc(override = "long", jupyter_display=True, precision=8)
def å2k(r):
  r_k = ((1 + r)**(1/4) - 1)
  return locals()

### Eksempel

Bestem den kvartalsvise rente, hvis den årlige rente er $3\%$

In [6]:
resultat = å2k(0.03)

<IPython.core.display.Latex object>

dvs, den kvartalsvise rente er 

In [7]:
f"{resultat['r_k']:.4%}"

'0.7417%'

## Annuitetsopsparing

Hvis man indbetaler det samme beløb $b$ hver termin, og rentefoden er $r$, så er kapitalen $A$ umiddelbart efter den $n$’te indbetaling:

$$A = b\frac{(1 + r )^n - 1 }{r}$$

In [8]:
@handcalc(override = "long", jupyter_display=True, precision=8)
def opsparing(b, r, n):
  b = b
  r = r
  n = n
  A = b * ((1+r)**n - 1)/r
  return locals()

### Eksempel

En familie vil spare op til deres drømmehus. De vil hver måned indbetale 6500 kr. Den månedlige rente er $0,25\%$. Bestem saldoen på familiens opsparing efter 10 år

In [9]:
res = opsparing(6500, 0.25/100, 4*30)

<IPython.core.display.Latex object>

## Gældsannuitet

Hvis man indbetaler ydelsen $y$ hver termin i $n$ terminer (første gang man betaler er én termin efter lånet er optaget), og rentefoden er $r$ , så er lånets størrelse:

$$G =  y \frac{1-(1+r)^{-n}}{r} = y \frac{1-   \frac{1}{(1+r)^n} }{r}  $$

In [10]:
@handcalc(override = "long", jupyter_display=True, precision=8)
def gæld(y, r, n):
  y = y
  r = r
  n = n
  A =  y * (1 - 1/(1+r)**n)/r
  return locals()

### Eksempel

En familie har råd til at betale et lån til en bolig tilbage med en ydelse på 20000 kr pr kvartal gennem 30 år. Bestem hvor stort et beløb familien kan låne, når den kvartalsvise rente er 0,74%

In [11]:
resultat = gæld(20000, 0.741707178/100, 4*30)

<IPython.core.display.Latex object>

In [12]:
20000 * 4 * 30 - resultat['A']

814432.9243761345

### Henvisninger

https://www.webmatematik.dk/lektioner/matematik-c/funktioner/opsparingsannuitet

https://www.webmatematik.dk/lektioner/matematik-c/funktioner/galdsannuitet

In [13]:
G* r/(1-(1+r)^-n)

NameError: name 'G' is not defined

In [14]:
r = 0.741707178/100

In [39]:
1585567.07562387 * r /(1-(1+r)**-120)

20000.00000000006

In [41]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [66]:
def amortize(principal, interest_rate, years, annual_payments):

    pmt = -round(np.pmt(interest_rate, years*annual_payments, principal), 2)
    # initialize the variables to keep track of the periods and running balances
    p = 1
    beg_balance = principal
    end_balance = principal

    while end_balance > 0:

        # Recalculate the interest based on the current balance
        interest = round((interest_rate * beg_balance), 2)

        # Determine payment based on whether or not this period will pay off the loan
        pmt = min(pmt, beg_balance + interest)

        principal = pmt - interest

        # Ensure additional payment gets adjusted if the loan is being paid off
        #end_balance = beg_balance - (principal + beg_balance - principal)
        end_balance = beg_balance - principal
                                     
        yield OrderedDict([('Periode', p),
                           ('Start hovedstol', beg_balance),
                           ('Ydelse', pmt),
                           ('Afdrag', principal),
                           ('Rente', interest),
                           ('Slut hovedstol', end_balance)])

        # Increment the counter, balance and date
        p += 1
        beg_balance = end_balance

In [69]:
schedule = pd.DataFrame(amortize(1585567.07562387, 0.00741707178, 30, 4))
schedule.tail()

<ipython-input-66-f3e3e1596a13>:3: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).
  pmt = -round(np.pmt(interest_rate, years*annual_payments, principal), 2)


,Periode,Start hovedstol,Ydelse,Afdrag,Rente,Slut hovedstol
115,116,97812.685624,20000.000000,19274.520000,725.48,78538.165624
116,117,78538.165624,20000.000000,19417.480000,582.52,59120.685624
117,118,59120.685624,20000.000000,19561.500000,438.50,39559.185624
118,119,39559.185624,20000.000000,19706.590000,293.41,19852.595624
119,120,19852.595624,19999.845624,19852.595624,147.25,0.000000


In [50]:
#np.pmt(interest_rate/annual_payments, years*annual_payments, principal)
np.pmt(0.00741707178, 30*4, 1585567.07562387)

<ipython-input-50-555fd4e5787b>:2: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).
  np.pmt(0.00741707178, 30*4, 1585567.07562387)


-20000.00000000006

In [68]:
schedule['Rente'].sum()

814432.77